In [1]:
import os
import numpy as np
import pandas as pd
from skimage.io import imread
from skimage.transform import resize
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50, ResNet152V2  # Import ResNet152V2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout, Flatten, BatchNormalization
from tensorflow.keras.optimizers import Adam
import tensorflow as tf
from sklearn.utils import class_weight
from sklearn.metrics import classification_report
from skimage.exposure import equalize_adapthist  # For histogram equalization
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

2023-08-27 05:02:16.043416: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-27 05:02:20.357509: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/nccl2/lib:/usr/local/cuda/extras/CUPTI/lib64
2023-08-27 05:02:20.359580: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/nccl2/lib:/usr/loca

In [2]:
# Load metadata CSV file
metadata = pd.read_csv('HAM10000_metadata.csv')

# Load images and labels
image_folder = 'HAM10000_Images'
image_size = (96, 96)


In [3]:
images = []
labels = []

for index, row in metadata.iterrows():
    image_path = os.path.join(image_folder, row['image_id'] + '.jpg')
    image = imread(image_path)
    resized_image = resize(image, image_size)
    images.append(resized_image)
    labels.append(row['dx'])


In [4]:
images = np.array(images)
labels = np.array(labels)

In [5]:
# Convert labels to categorical format
label_mapping = {label: idx for idx, label in enumerate(np.unique(labels))}
labels_encoded = np.array([label_mapping[label] for label in labels])
labels_categorical = to_categorical(labels_encoded, num_classes=len(label_mapping))

In [6]:
# Split the dataset into train, validation, and test sets
train_images, test_images, train_labels, test_labels = train_test_split(
    images, labels_categorical, test_size=0.2, random_state=42
)
train_images, val_images, train_labels, val_labels = train_test_split(
    train_images, train_labels, test_size=0.2, random_state=42
)


In [7]:

# Print the shape of each set
print("Train images shape:", train_images.shape)
print("Train labels shape:", train_labels.shape)
print("Validation images shape:", val_images.shape)
print("Validation labels shape:", val_labels.shape)
print("Test images shape:", test_images.shape)
print("Test labels shape:", test_labels.shape)


Train images shape: (6409, 96, 96, 3)
Train labels shape: (6409, 7)
Validation images shape: (1603, 96, 96, 3)
Validation labels shape: (1603, 7)
Test images shape: (2003, 96, 96, 3)
Test labels shape: (2003, 7)


In [13]:
# Create an ImageDataGenerator instance for data augmentation
datagen = ImageDataGenerator(
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode='nearest',
    preprocessing_function=equalize_adapthist  # Apply histogram equalization
)

In [14]:
# Apply data augmentation only to the training set
train_datagen = datagen.flow(train_images, train_labels, batch_size=32, shuffle=True)

ResNet152V2

In [24]:

# Load the pre-trained ResNet152V2 model
base_model1 = ResNet152V2(include_top=False, weights='imagenet', input_shape=(96, 96, 3))


In [26]:
# Fine-tune the model
# fine_tune_at = 555
# for layer in base_model1.layers[:fine_tune_at]:
#     layer.trainable = False
for layer in base_model1.layers:
    layer.trainable = False

In [27]:
# Implement your specified model
model1 = Sequential()
model1.add(base_model1)
model1.add(Flatten())
model1.add(BatchNormalization())
model1.add(Dense(128, activation='relu'))
model1.add(BatchNormalization())
model1.add(Dropout(0.5))
model1.add(Dense(7, activation='softmax'))
print(model1.summary())

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet152v2 (Functional)    (None, 3, 3, 2048)        58331648  
                                                                 
 flatten_1 (Flatten)         (None, 18432)             0         
                                                                 
 batch_normalization_4 (Batc  (None, 18432)            73728     
 hNormalization)                                                 
                                                                 
 dense_6 (Dense)             (None, 128)               2359424   
                                                                 
 batch_normalization_5 (Batc  (None, 128)              512       
 hNormalization)                                                 
                                                                 
 dropout_3 (Dropout)         (None, 128)              

EfficientNetB7

In [19]:
from tensorflow.keras.applications import VGG19

base_model3 = VGG19(include_top = False,input_shape=(96, 96, 3),weights='imagenet')
# Fine-tune the model
fine_tune_at = 20
for layer in base_model3.layers[:fine_tune_at]:
    layer.trainable = False

model3 = Sequential()
model3.add(base_model2)
model3.add(GlobalAveragePooling2D())
model3.add(Dense(128, activation='relu'))
model3.add(BatchNormalization())
model3.add(Dropout(0.5))
model3.add(Dense(7, activation='softmax'))

In [20]:
# Learning rate scheduling
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-6)

In [21]:
# Early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

Model 1 resnet152v2

In [28]:

# Compile the model
model1.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

In [32]:
# Train the model with the improved settings
epochs = 40
batch_size = 32

history = model1.fit(
    train_datagen,
    batch_size=batch_size,
    epochs=epochs,
    validation_data=(val_images, val_labels),
    callbacks=[early_stopping, reduce_lr]
)


Epoch 1/40
201/201 [==============================] - 348s 2s/step - loss: 2.1297 - accuracy: 0.3848 - val_loss: 1.7415 - val_accuracy: 0.5384 - lr: 1.0000e-04
Epoch 2/40
201/201 [==============================] - 339s 2s/step - loss: 1.8646 - accuracy: 0.4803 - val_loss: 1.5173 - val_accuracy: 0.6095 - lr: 1.0000e-04
Epoch 3/40
201/201 [==============================] - 340s 2s/step - loss: 1.6814 - accuracy: 0.5339 - val_loss: 1.4377 - val_accuracy: 0.6369 - lr: 1.0000e-04
Epoch 4/40
201/201 [==============================] - 341s 2s/step - loss: 1.6040 - accuracy: 0.5494 - val_loss: 1.2885 - val_accuracy: 0.6600 - lr: 1.0000e-04
Epoch 5/40
201/201 [==============================] - 342s 2s/step - loss: 1.4695 - accuracy: 0.5826 - val_loss: 1.2894 - val_accuracy: 0.6619 - lr: 1.0000e-04
Epoch 6/40
201/201 [==============================] - 338s 2s/step - loss: 1.3781 - accuracy: 0.6104 - val_loss: 1.2367 - val_accuracy: 0.6744 - lr: 1.0000e-04
Epoch 7/40
201/201 [====================

In [39]:
# Evaluate the model on the validation set
val_loss, val_accuracy = model1.evaluate(val_images, val_labels, batch_size=batch_size)
print("Validation Loss:", val_loss)
print("Validation Accuracy:", val_accuracy)

51/51 [==============================] - 62s 1s/step - loss: 0.9645 - accuracy: 0.7037
Validation Loss: 0.964523434638977
Validation Accuracy: 0.7036805748939514


In [41]:
# Evaluate the model on the testing set
test_loss, test_accuracy = model1.evaluate(test_images, test_labels, batch_size=batch_size)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)


63/63 [==============================] - 81s 1s/step - loss: 0.9838 - accuracy: 0.7109
Test Loss: 0.9837701320648193
Test Accuracy: 0.7109336256980896


In [42]:

# Generate a classification report
y_pred = model1.predict(test_images)
y_pred = np.argmax(y_pred, axis=1)
print(classification_report(np.argmax(test_labels, axis=1), y_pred))



63/63 [==============================] - 80s 1s/step
              precision    recall  f1-score   support

           0       0.40      0.06      0.10        69
           1       0.49      0.28      0.36        93
           2       0.50      0.26      0.34       228
           3       0.00      0.00      0.00        28
           4       0.48      0.22      0.30       226
           5       0.75      0.96      0.84      1338
           6       0.17      0.05      0.07        21

    accuracy                           0.71      2003
   macro avg       0.40      0.26      0.29      2003
weighted avg       0.65      0.71      0.66      2003



In [43]:
# Save the model for later use in your web app
model1.save('cnn_model-resnet152v2_96x96v2')

INFO:tensorflow:Assets written to: cnn_model-resnet152v2_96x96v2/assets


INFO:tensorflow:Assets written to: cnn_model-resnet152v2_96x96v2/assets
